# # Screening Ecoinvent market consumption mixes: grouping by markets

In [24]:
import brightway2 as bw
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
import presamples as ps
import scipy
import scipy.stats as stats
from code.functions import make_dirichlet_distribution, get_elec_input, create_presamples, query_for_activities
ps.__version__

(0, 2, 6)

In [25]:
from presamples import array

def custom_sample(self, index):
    """Draw a new sample from the pre-sampled arrays sequentially not randomly. The original code draws samples randomly. """
    index_ordered = self.count % self.data[0].shape[1]
    result = np.hstack([arr[:, index_ordered] for arr in self.data])
    self.count += 1   
    return result


array.RegularPresamplesArrays.sample = custom_sample

In [26]:
bw.projects.set_current("UK-wood-clca")

cutoff391 = bw.Database('cutoff-3.9.1')
fus = pd.read_excel(r"run.xlsx", sheet_name='to_run')
numbers = fus['indices'].tolist()
output_dir  = fus['names'].tolist()
amount  = fus['amount'].tolist()

methods = [

     ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)'), 

]

In [27]:
output_dir 

['_market_for_sawnwood__board__softwood__raw__dried__u_20_____cubic_meter__CH__None_']

In [28]:
i = 0

In [29]:
from code.morris_market_analysis import perform_morris_market_analysis

In [30]:
    category_mapping = {
    "01": "Crop and animal production, hunting and related service activities",
    "02": "Forestry and logging",
    "03": "Fishing and aquaculture",
    "05": "Mining of coal and lignite",
    "06": "Extraction of crude petroleum and natural gas",
    "07": "Mining of metal ores",
    "08": "Other mining and quarrying",
    "09": "Mining support service activities",
    "10": "Manufacture of food products",
    "11": "Manufacture of beverages",
    "12": "Manufacture of tobacco products",
    "13": "Manufacture of textiles",
    "14": "Manufacture of wearing apparel",
    "15": "Manufacture of leather and related products",
    "16": "Manufacture of wood and products of wood and cork",
    "17": "Manufacture of paper and paper products",
    "18": "Printing and reproduction of recorded media",
    "19": "Manufacture of coke and refined petroleum products",
    "20": "Manufacture of chemicals and chemical products",
    "21": "Manufacture of pharmaceuticals, medicinal chemical and botanical products",
    "22": "Manufacture of rubber and plastic products",
    "23": "Manufacture of other non-metallic mineral products",
    "24": "Manufacture of basic metals",
    "25": "Manufacture of fabricated metal products",
    "26": "Manufacture of computer, electronic, and optical products",
    "27": "Manufacture of electrical equipment",
    "28": "Manufacture of machinery and equipment",
    "29": "Manufacture of motor vehicles, trailers, and semi-trailers",
    "30": "Manufacture of other transport equipment",
    "31": "Manufacture of furniture",
    "32": "Other manufacturing",
    "33": "Repair and installation of machinery and equipment",
    "35": "Electricity, gas, steam and air conditioning supply",
    "36": "Water collection, treatment, and supply",
    "37": "Sewerage",
    "38": "Waste collection, treatment, and disposal activities",
    "39": "Remediation activities and other waste management services",
    "41": "Construction of buildings",
    "42": "Civil engineering",
    "43": "Specialized construction activities",
    "45": "Wholesale and retail trade and repair of motor vehicles and motorcycles",
    "46": "Wholesale trade, except of motor vehicles and motorcycles",
    "47": "Retail trade, except of motor vehicles and motorcycles",
    "49": "Land transport and transport via pipelines",
    "50": "Water transport",
    "51": "Air transport",
    "52": "Warehousing and support activities for transportation",
    "53": "Postal and courier activities",
    "55": "Accommodation",
    "56": "Food and beverage service activities",
    "58": "Publishing activities",
    "59": "Motion picture, video and television programme production, sound recording, and music publishing activities",
    "60": "Programming and broadcasting activities",
    "61": "Telecommunications",
    "62": "Computer programming, consultancy, and related activities",
    "63": "Information service activities",
    "68": "Real estate activities",
    "69": "Legal and accounting activities",
    "70": "Activities of head offices, management consultancy activities",
    "71": "Architectural and engineering activities, technical testing and analysis",
    "72": "Scientific research and development",
    "73": "Advertising and market research",
    "74": "Other professional, scientific, and technical activities",
    "75": "Veterinary activities",
    "77": "Rental and leasing activities",
    "78": "Employment activities",
    "79": "Travel agency, tour operator, and other reservation service and related activities",
    "80": "Security and investigation activities",
    "81": "Services to buildings and landscape activities",
    "82": "Office administrative, office support, and other business support activities",
    "84": "Public administration and defence, compulsory social security",
    "85": "Education",
    "86": "Human health activities",
    "87": "Residential care activities",
    "88": "Social work activities without accommodation",
    "90": "Creative, arts and entertainment activities",
    "91": "Libraries, archives, museums, and other cultural activities",
    "92": "Gambling and betting activities",
    "93": "Sports activities and amusement and recreation activities",
    "94": "Activities of membership organizations",
    "95": "Repair of computers and personal and household goods",
    "96": "Other personal service activities",
    "97": "Activities of households as employers of domestic personnel",
    "98": "Undifferentiated goods- and services-producing activities of households for own use",
    "99": "Activities of extraterritorial organizations and bodies"
}


In [31]:
Global_cat= pd.read_csv("results/Global__market_categories_4_products_to_detailed_analysis.csv")
reverse_mapping = {v: k for k, v in category_mapping.items()}

Global_cat['Category Number'] = Global_cat['names'].map(reverse_mapping)

fus = pd.read_excel(r"run.xlsx", sheet_name='to_run')
numbers = fus['indices'].tolist()
output_dir  = fus['names'].tolist()
amount  = fus['amount'].tolist()
amount  = fus['amount'].tolist()
reference_product  = fus['reference_product'].tolist()
tier = fus['tier'].tolist()

fu = [cutoff391.get({num}) for num in numbers]


In [32]:
Global_cat['fu'][0]

'_market_for_sawnwood__board__softwood__raw__dried__u_20_____cubic_meter__CH__None_'

In [33]:
for i in range(len(output_dir)):
     print(len(Global_cat[Global_cat['fu'] == output_dir[i]]['names'].values))

4


In [34]:
output_dir

['_market_for_sawnwood__board__softwood__raw__dried__u_20_____cubic_meter__CH__None_']

In [35]:
%%time
levels = [12]
Global_market_scores = []

for i in range(len(output_dir)): 
    if len(Global_cat[Global_cat['fu'] == output_dir[i]]['names'].values) != 0:
        for level in levels:
            sa_market = perform_morris_market_analysis(
                fu[i], 
                1,
                output_dir[i],
                output_dir[i], 
                methods, 
                cutoff391, 
                trajectories=50,  
                num_levels=level,
                category_names=Global_cat[Global_cat['fu'] == output_dir[i]]['names'].values,
                category_numbers=Global_cat[Global_cat['fu'] == output_dir[i]]['Category Number'].values,
                mean =117.8113726,# 2.870307461,
                market_dir=output_dir[i],
            )
            Global_market_scores.append(sa_market)
Global_market_scores = pd.concat(Global_market_scores, ignore_index=True)
#Global_market_scores.to_csv(f"results/Global_scores_by_market_4_other_products.csv", index=False)


188


C:\Users\rla44\OneDrive\Github\GSA_markets\code\morris_market_analysis.py:347: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  yield np.hstack(compositions)


2069
2069
1881
CPU times: total: 27min 8s
Wall time: 24min 35s


In [36]:

#Global_market_scores.to_csv(f"results/Global_scores_by_market_4_other_products_2.csv", index=False)